# MVP Engenharia de Dados

**Nome:** Luis Cláudio da Paixão Lobato

**Matrícula:** 4052025001146

**Dataset:** [TISS] (https://dadosabertos.ans.gov.br/FTP/PDA/TISS/)

# 1️⃣ Objetivo
___

### 1.1 Descrição do MVP ###

Este MVP tem como objetivo construir um pipeline de dados na nuvem para analisar os gastos com atendimento ambulatorial e internação registrados no padrão TISS (Troca de Informações em Saúde Suplementar) para o estado de Goiás, no período de 2020 a 2024. A solução será implementada com tecnologias em nuvem, utilizando Databricks e Delta Lake, e abrangerá desde a ingestão até a disponibilização de dados prontos para análise.

### 1.2 Problema central ###

Atualmente não existe uma visão consolidada e facilmente acessível dos custos de internação e atendimento ambulatorial em Goiás, o que dificulta a identificação de tendências, anomalias e fatores relevantes para a gestão da saúde pública. Este MVP busca preencher essa lacuna, oferecendo uma base confiável que suporte análises operacionais e estratégicas.


### 1.3 Escopo e etapas do pipeline ###

- Busca e coleta: extração dos arquivos TISS e fontes complementares para o período 2020–2024.
- Ingestão e armazenamento: carregamento em Delta Lake para garantir governança, versionamento e performance.
- Modelagem: padronização, limpeza e harmonização dos dados (identificadores, datas, valores, procedimentos).
- Carga e transformação: criação de camadas (bronze, silver, gold) para suportar diferentes níveis de consumo.
- Análise e visualização: geração de métricas, dashboards e relatórios para identificar padrões de gasto, outliers e drivers de custo.

### 1.4 Perguntas que o projeto pretende responder ###

- Quais são as tendências temporais de gasto por tipo de atendimento (ambulatorial vs internação)?
- Quais procedimentos, especialidades ou prestadores concentram os maiores custos?
- Existem anomalias ou picos de gasto que mereçam investigação?
- Quais fatores (idade, região, tipo de procedimento) mais impactam os custos?

### 1.5 Entregáveis esperados ###

- Base de dados consolidada em Delta Lake cobrindo 2020–2024.
- Conjunto de pipelines automatizados para ingestão, transformação e atualização dos dados.
- Dashboards e relatórios analíticos com indicadores-chave e insights acionáveis.
- Documentação técnica e guia de uso para replicação e manutenção.
Ao final do projeto, espera-se dispor de uma plataforma confiável que permita à gestão pública e a analistas compreender melhor os determinantes dos custos em saúde no estado de Goiás e apoiar decisões baseadas em dados.



# 2️⃣ Fonte dos Dados e Coleta
___

Os dados utilizados neste projeto foram obtidos exclusivamente de fontes oficiais e públicas, portanto não há restrições de confidencialidade. A base principal é a Troca de Informações em Saúde Suplementar (TISS), que reúne registros detalhados sobre atendimentos ambulatoriais e internações em todo o Brasil. O conjunto de dados abrange o período de 2020 a 2024 e inclui informações relevantes para análise de custos, procedimentos, prestadores e características demográficas dos atendimentos.

### 2.1 Tabela Fato – Internação e Atendimento Ambulatorial 

As tabelas fato do projeto — `internacao_geral_gold` e `ambulatorial_geral_gold` — foram geradas a partir dos arquivos de internação e de atendimento ambulatorial disponibilizados no portal de dados abertos da ANS. 

Fonte: Troca de Informações em Saúde Suplementar (TISS) — https://dadosabertos.ans.gov.br/FTP/PDA/TISS/.

Os arquivos foram baixados manualmente, mês a mês, no formato CSV, e posteriormente carregados no DBFS (Databricks File System). A partir desses arquivos foram aplicados processos de limpeza, padronização e integração para compor as camadas do Delta Lake e consolidar as tabelas gold para análise.

### 2.2 Tabelas Dimensão

As tabelas dimensão foram obtidas no portal da ANS, e criados manualmente. Foram criadas no formato CSV, separadas por ";", utilizando um editor de texto. Essas tabelas foram baseadas diretamente nas informações contidas na tabela fato `internacao_geral_gold` e `ambulatorial_geral_gold` e incluem:

-

-

-

-

-

-

-

-

-

 Essas tabelas foram projetadas para servir de referência às respectivas colunas nas bases de internação e atendimento ambulatorial, garantindo a integridade dos dados no processo analítico.





# 3️⃣ Modelagem e Catálogo de Dados
___

Para estruturar e organizar os dados de forma eficiente, foi adotado o Esquema Estrela, um dos modelos mais utilizados em Data Warehousing e Business Intelligence.

### 3.1 Estrutura do Esquema Estrela

O esquema estrela do projeto foi construído com uma tabela fato principal contendo os registros de mortalidade e 7 tabelas dimensão para compor as análises. A estrutura ficou organizada da seguinte forma:

 📊 Tabela Fato: `mortalidade_geral_gold`
  
  - Esta tabela contém os registros de óbitos e é o núcleo central do esquema. Cada linha representa um óbito registrado, com detalhes como data, local, causa da morte e características da pessoa falecida.

  📊 Tabelas Dimensão: `municipios_gold`, `cid_10_gold`, `circunstancia_gold`, `local_obito_gold`, `estado_civil_gold`, `sexo_gold` e `cor_gold`.

  - Foram criadas tabelas auxiliares para armazenar descrições de variáveis categóricas e facilitar a análise por meio de junções (joins) entre as tabelas.

### 3.2 Catálogo de Dados

### 3.3 Diagrama Entidade Relacionamento

# 4️⃣ Carga
___

Nesta etapa será feita a carga dos dados para o Delta Lake do Databricks. Será feito um pipeline de ETL (Extração, Transformação e Carga) e todos os processos realizados serão documentados.

Inicialmente, importaremos as bibliotecas necessárias e criaremos o banco de dados, organizando os dados em três camadas dentro da arquitetura medallion (Bronze, Silver e Gold). Cada camada terá um papel na preparação dos dados:

🥉 Camada Bronze: Armazena os dados brutos, exatamente como foram extraídos das fontes originais, sem qualquer modificação.

🥈 Camada Silver: Aplica limpeza, padronização e enriquecimento dos dados, removendo inconsistências e garantindo a qualidade.

🥇 Camada Gold: Otimiza as tabelas para consultas analíticas e geração de insights para consumo final.

### 💾 4.1 Banco de dados

In [ ]:
from pyspark.sql.functions import col, when, concat, substring, lit, to_date  # Para manipular dados em spark
from pyspark.sql import Row  # Para manipular dados em spark
import matplotlib.pyplot as plt  # Para plotar gráficos
import seaborn as sns  # Para plotar gráficos
import pandas as pd  # Para criar dataframes que possam ser lidos pelo matplotlib e seaborn
import warnings  # Para desativar warnings
warnings.filterwarnings("ignore")

Vamos criar 3 bancos de dados para separar nosso projeto: `bronze`, `silver` e `gold`. Iniciaremos guardando os dados crus no banco de dados `bronze`.

### 🥉 4.2 Camada Bronze